In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Keep every_ith control. Increase this if you want to decrease the number of controls being considered.
every_ith = 10
import os
import dill
import numpy as np
mypath = 'controls_106'
_file = 'pickled_controls106.pkl'
filename = os.path.join(mypath, _file)
pca = dill.load(open(filename, 'rb'))

def bangbang(index, x, d1, d2, d3, d4):
    if x:
        xory = 0
    else:
        xory = 1
    dt = pca.dt
    num_steps = pca.controlset[0].shape[0]
    control_hamiltonians = pca.control_hamiltonians
    controls = np.zeros((num_steps, int(len(pca.control_hamiltonians))))
    controls[index, xory] = (1 / dt * np.pi / 2) * d1
    controls[-(index+1), xory] = 1 / dt * np.pi / 2 * d2
    controls[index, 2+xory] = 1 / dt * np.pi / 2 * d3
    controls[-(index+1), 2+xory] = 1 / dt * np.pi / 2 * d4
    return controls

controls = []
import itertools
for directions in list(itertools.product([-1, 1], repeat=4)):
    for x in (True, False):
        for i in range(int(pca.controlset[0].shape[0]/2)):
            if i % every_ith != 0:
                continue
            controls.append(bangbang(i, x, *directions))
pca.controlset = controls
pca.num_controls = len(pca.controlset)
pca.weights = None

In [4]:
from convex import all_derivs
derivs = all_derivs(pca.controlset, pca.target_operator, pca.control_hamiltonians, pca.ambient_hamiltonian, pca.dt, 1)

In [5]:
from pauli_channel_approximation import PCA

In [141]:
from convex import optimal_weights
pca.weights_0 = optimal_weights(derivs[:1], 1E-4)

Done with convex problem 0 out of 64
Done with convex problem 10 out of 64
Done with convex problem 20 out of 64
Done with convex problem 30 out of 64
Done with convex problem 40 out of 64
Done with convex problem 50 out of 64
Done with convex problem 60 out of 64


In [142]:
from convex import optimal_weights
pca.weights = optimal_weights(derivs, 1E-4)

Done with convex problem 0 out of 64
Done with convex problem 10 out of 64
Done with convex problem 20 out of 64
Done with convex problem 30 out of 64
Done with convex problem 40 out of 64
Done with convex problem 50 out of 64
Done with convex problem 60 out of 64


In [143]:
import matplotlib.pyplot as plt
def hist_and_support(weights, num=10):
    plt.hist(weights)
    np.argmax(weights)
    top_idx = np.argsort(np.array(weights).T)[0][-num:]
    top_values = [weights[i] for i in top_idx]
    print(f"SUPPORT:{np.sum(top_values)}")
    print(f"NUM CONTROLS: {num}")
    return top_idx, np.sum(top_values)

In [144]:
num_weights = 20

In [145]:
idx0, supp = hist_and_support(pca.weights_0, num_weights)

SUPPORT:0.9999999937233088
NUM CONTROLS: 20


In [146]:
pca0 = dill.load(open(filename, 'rb'))
subweights = pca.weights_0[idx0]
pca0.controlset = [pca0.controlset[i] for i in idx0]
pca0.probs = list(np.array(subweights.T/sum(subweights)).T)

In [147]:
import scipy
order = 0
# Zeroth order optimized weights
scipy.linalg.norm(np.matrix(np.array([d.flatten() for d in derivs[order]])[idx0]).T.dot(pca0.probs))

2.6711914707529945e-10

In [148]:
import scipy
order = 1
# Zeroth order optimized weights
scipy.linalg.norm(np.matrix(np.array([d.flatten() for d in derivs[order]])[idx0]).T.dot(pca0.probs))

2.1878441028807805

In [149]:
idx1, supp = hist_and_support(pca.weights, num_weights)

SUPPORT:0.9999988742973783
NUM CONTROLS: 20


In [150]:
pca1 = dill.load(open(filename, 'rb'))
subweights = pca.weights[idx1]
pca1.controlset = [pca1.controlset[i] for i in idx1]
pca1.probs = list(np.array(subweights.T/sum(subweights)).T)

In [151]:
import scipy
order = 0
# Zeroth order optimized weights
scipy.linalg.norm(np.matrix(np.array([d.flatten() for d in derivs[order]])[idx1]).T.dot(pca1.probs))

3.5393445382840956e-08

In [152]:
import scipy
order = 1
# Zeroth order optimized weights
scipy.linalg.norm(np.matrix(np.array([d.flatten() for d in derivs[order]])[idx1]).T.dot(pca1.probs))

0.01641236607906174

In [153]:
# Save the weighted controls
import dill
dill.dump(pca0, open('0_' + _file, 'wb'))

import dill
dill.dump(pca1, open('1_' + _file, 'wb'))

In [154]:
# Save the weighted controls
import dill
import os
mypath = 'controls_106'
_file = 'pickled_controls106.pkl'
filename = os.path.join(mypath, _file)
pca = dill.load(open(filename, 'rb'))

pca0 = dill.load(open('0_' + _file, 'rb'))
pca1 = dill.load(open('1_' + _file, 'rb'))

In [155]:
from pauli_channel_approximation import control_unitaries, error_unitary, off_diagonal_projection, adjoint
import sys
import itertools
from itertools import product
from copy import deepcopy
from functools import reduce
import numpy as np

from diamond import diamond_norm, jamiolkowski

def superoperator(unitary):

    return np.kron(unitary.conj(), unitary)

def choi_form(unitary):
    return np.matrix(jamiolkowski(superoperator(unitary)))

def diamond_distance(sopa, sopb):
#     unitary_a = np.matrix(unitary_a)
#     unitary_b = np.matrix(unitary_b)
#     arg = (choi_form(unitary_a) - choi_form(unitary_b))/2
#     rtn = diamond_norm(arg)
    rtn = (jamiolkowski(sopa) - jamiolkowski(sopb))/2
    return diamond_norm(rtn)

def compute_diamond_norm(data):
    controlset, ambient_hamiltonian0, combo, dt, control_hamiltonians, target_operator, probs = data
    print("DOING COMBO {}".format(combo))
    sys.stdout.flush()
    fidelities = []
    projs = []
    sops = []
    controlset_unitaries = []
    d_norms = []

    for controls in controlset:
        newcontrols = deepcopy(controls)
        ambient_hamiltonian = [deepcopy(ah).astype("complex") for ah in ambient_hamiltonian0]
        for cnum, value in enumerate(combo):
            cnum -= len(ambient_hamiltonian0)
            if cnum >= 0:
                newcontrols[:, cnum] = newcontrols[:, cnum] * (1 + value)
            if cnum < 0:
                if cnum == -1:
                    ambient_hamiltonian[cnum] *= 1 + float(value)
                else:
                    ambient_hamiltonian[cnum] *= float(value)
        step_unitaries = control_unitaries(ambient_hamiltonian,
                                           control_hamiltonians, newcontrols,
                                           dt)
        unitary = reduce(lambda a, b: a.dot(b), step_unitaries)
        sop = error_unitary(unitary, target_operator)
        sops.append(sop)
        d_norms.append(diamond_distance(sop, np.eye(sop.shape[0])))
    avg_sop = reduce(lambda a, b: a + b, [prob * sops[i] for i, prob in enumerate(probs)])
    d_norms.append(diamond_distance(avg_sop, np.eye(avg_sop.shape[0])))
    return d_norms

def generate_indices(num_points, order_desired):
    num_indices = len(order_desired)
    tuples = product(range(num_points), repeat=num_indices)
    indices = [sum([num_points**(num_indices - 1 - order_desired[i]) * t[i] 
                    for i in range(num_indices)]) for t in tuples]
    return indices

In [158]:
import dill
all_results = []
for j, pca in enumerate((pca0, pca1)):
    base = 100*np.logspace(-4.7, np.log(pca.detunings[0][0])/np.log(10), 1)
    controls = np.concatenate([np.sort(-base),np.array([0]), base])
    combinations0 = [[det, 0, 0, 0, 0, 0, 0] for det in controls]
    combinations1 = [[0, det, 0, 0, 0, 0, 0] for det in controls]
    combinations2 = [[0, 0, det, 0, 0, 0, 0] for det in controls]
    combinations3 = [[0, 0, 0, det, det, 0, 0] for det in controls]
    combinations4 = [[0, 0, 0, 0, 0, det, det] for det in controls]

    d_norms00 = []
    for i in range(len(combinations0)):
        combo = combinations0[i]
        lst = [pca.controlset, pca.ambient_hamiltonian, combo, pca.dt,
                pca.control_hamiltonians, pca.target_operator, pca.probs]
        d_norms00.append(compute_diamond_norm(lst))
        dill.dump(d_norms00, open('d_norms00_{}_{}'.format(i, j), 'wb'))

    d_norms01 = []
    for i in range(len(combinations1)):
        combo = combinations1[i]
        lst = [pca.controlset, pca.ambient_hamiltonian, combo, pca.dt,
                pca.control_hamiltonians, pca.target_operator, pca.probs]
        d_norms01.append(compute_diamond_norm(lst))
        dill.dump(d_norms01, open('d_norms01_{}_{}'.format(i, j), 'wb'))

    d_norms02 = []
    for i in range(len(combinations2)):
        combo = combinations2[i]
        lst = [pca.controlset, pca.ambient_hamiltonian, combo, pca.dt,
                pca.control_hamiltonians, pca.target_operator, pca.probs]
        d_norms02.append(compute_diamond_norm(lst))
        dill.dump(d_norms02, open('d_norms02_{}_{}'.format(i, j), 'wb'))

    d_norms03 = []
    for i in range(len(combinations3)):
        combo = combinations3[i]
        lst = [pca.controlset, pca.ambient_hamiltonian, combo, pca.dt,
                pca.control_hamiltonians, pca.target_operator, pca.probs]
        d_norms03.append(compute_diamond_norm(lst))
        dill.dump(d_norms03, open('d_norms03_{}_{}'.format(i, j), 'wb'))

    d_norms04 = []
    for i in range(len(combinations4)):
        combo = combinations4[i]
        lst = [pca.controlset, pca.ambient_hamiltonian, combo, pca.dt,
                pca.control_hamiltonians, pca.target_operator, pca.probs]
        d_norms04.append(compute_diamond_norm(lst))
        dill.dump(d_norms04, open('d_norms04_{}_{}'.format(i, j), 'wb'))
        
    all_results.extend([d_norms00, d_norms01, d_norms02, d_norms03, d_norms04])
dill.dump(all_results, open('all_results', 'wb'))

DOING COMBO [-0.0019952623149688785, 0, 0, 0, 0, 0, 0]
DOING COMBO [0.0, 0, 0, 0, 0, 0, 0]
DOING COMBO [0.0019952623149688785, 0, 0, 0, 0, 0, 0]
DOING COMBO [0, -0.0019952623149688785, 0, 0, 0, 0, 0]
DOING COMBO [0, 0.0, 0, 0, 0, 0, 0]
DOING COMBO [0, 0.0019952623149688785, 0, 0, 0, 0, 0]
DOING COMBO [0, 0, -0.0019952623149688785, 0, 0, 0, 0]
DOING COMBO [0, 0, 0.0, 0, 0, 0, 0]
DOING COMBO [0, 0, 0.0019952623149688785, 0, 0, 0, 0]
DOING COMBO [0, 0, 0, -0.0019952623149688785, -0.0019952623149688785, 0, 0]
DOING COMBO [0, 0, 0, 0.0, 0.0, 0, 0]
DOING COMBO [0, 0, 0, 0.0019952623149688785, 0.0019952623149688785, 0, 0]
DOING COMBO [0, 0, 0, 0, 0, -0.0019952623149688785, -0.0019952623149688785]
DOING COMBO [0, 0, 0, 0, 0, 0.0, 0.0]
DOING COMBO [0, 0, 0, 0, 0, 0.0019952623149688785, 0.0019952623149688785]
DOING COMBO [-0.0019952623149688785, 0, 0, 0, 0, 0, 0]
DOING COMBO [0.0, 0, 0, 0, 0, 0, 0]
DOING COMBO [0.0019952623149688785, 0, 0, 0, 0, 0, 0]
DOING COMBO [0, -0.0019952623149688785, 0, 0

In [24]:
from pygsti.tools.gatetools import diamonddist

In [25]:
diamonddist()

<function pygsti.tools.gatetools.diamonddist(A, B, mxBasis='gm', return_x=False)>

In [26]:
x = np.array([[0, 1], [1, 0]])

In [27]:
i = np.eye(2)

In [30]:
choi_form(x)

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [36]:
diamond_distance(superoperator(x), superoperator(i))

0.999999993275624